In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-csv_2.10:1.3.0 pyspark-shell'

In [2]:
import pyspark
sc = pyspark.SparkContext()

In [3]:
print(sc)

In [4]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [5]:
# do something to prove it works
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 10)

[170, 633, 147, 710, 315, 16, 697, 44, 844, 413]

In [6]:
%pwd

'/home/jovyan/work'

In [7]:
%ls

Dockerfile  README.md  Untitled.ipynb


In [8]:
%cd /data

/data


In [9]:
%ls

enron_mail.seq


In [10]:
data = sc.sequenceFile('/data/enron_mail.seq')

In [11]:
print(data)

MapPartitionsRDD[5] at mapPartitions at SerDeUtil.scala:206


In [12]:
#data.count()

In [13]:
subset = data.top(10)

In [14]:
for record in subset:
    print(record[0])
    print(record[1])
    print

maildir/zufferli-j/sent_items/99.
Message-ID: <28618979.1075842030037.JavaMail.evans@thyme>
Date: Mon, 26 Nov 2001 10:48:43 -0800 (PST)
From: john.zufferli@enron.com
To: livia_zufferli@monitor.com
Subject: RE: ali's essays
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Zufferli, John </O=ENRON/OU=NA/CN=RECIPIENTS/CN=JZUFFER>
X-To: 'Livia_Zufferli@Monitor.com@ENRON'
X-cc: 
X-bcc: 
X-Folder: \ExMerge - Zufferli, John\Sent Items
X-Origin: ZUFFERLI-J
X-FileName: john zufferli 6-26-02.PST

i think the YMCA has a class that is for people recovering from heart-attacks
i remeber something about that

 -----Original Message-----
From: 	Livia_Zufferli@Monitor.com@ENRON  
Sent:	Monday, November 26, 2001 11:44 AM
To:	Zufferli, John
Subject:	RE: ali's essays


i don't know about the heart classes.  i'll look into it, but her dr (ravi)
isn't offering up any suggestions or anything.  she saw him before the
surgery in august, and he said things wer

In [15]:
import nltk

In [16]:
text = "this's a sent tokenize test. this is sent two. is this sent three? sent 4 is cool! Now it's your turn."
from nltk.tokenize import sent_tokenize
sent_tokenize_list = sent_tokenize(text)
len(sent_tokenize_list)

5

In [17]:
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences = tokenizer.tokenize(text)
print(sentences)

["this's a sent tokenize test.", 'this is sent two.', 'is this sent three?', 'sent 4 is cool!', "Now it's your turn."]


In [18]:
from nltk.tokenize import TreebankWordTokenizer

tokenizer = TreebankWordTokenizer()

for sentence in sentences:
    print(tokenizer.tokenize(sentence))

['this', "'s", 'a', 'sent', 'tokenize', 'test', '.']
['this', 'is', 'sent', 'two', '.']
['is', 'this', 'sent', 'three', '?']
['sent', '4', 'is', 'cool', '!']
['Now', 'it', "'s", 'your', 'turn', '.']


In [19]:
import nltk.data
from nltk.tokenize import TreebankWordTokenizer

from collections import namedtuple

fields = ('token', 'tokennumber', 'sentencenumber', 'textid')
Token = namedtuple('Token', fields)

def tokenize_text(row):
    text = row[1]
    textid = row[0]
  
    sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    word_tokenizer = TreebankWordTokenizer()
    
    tokens = []
    
    for i, s in enumerate(sent_tokenizer.tokenize(text)):
        for j, token in enumerate(word_tokenizer.tokenize(s)):
            #print('token: {}\tsentence: {}\tword: {}'.format(token, i, j))
            tokens.append(Token(token, j, i, textid))
    return tokens
            

for record in subset:
    ts = tokenize_text(record)

In [20]:
print(ts[6])

Token(token='>', tokennumber=6, sentencenumber=0, textid='maildir/zufferli-j/sent_items/90.')


In [21]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

schema = StructType([StructField('token', StringType(), True),
          StructField('tokennumber', IntegerType(), True),
          StructField('sentencenumber', IntegerType(), True),
          StructField('textid', StringType(), True)])

df = sqlContext.createDataFrame(sc.emptyRDD(), schema)
for record in subset:
    ts = tokenize_text(record)
    t = sqlContext.createDataFrame(ts)
    print(t.count())
    df = df.unionAll(t)

784
107
154
105
138
236
196
142
411
115


In [22]:
df.count()

2388

In [23]:
df.take(4)

[Row(token='Message-ID', tokennumber=0, sentencenumber=0, textid='maildir/zufferli-j/sent_items/99.'),
 Row(token=':', tokennumber=1, sentencenumber=0, textid='maildir/zufferli-j/sent_items/99.'),
 Row(token='<', tokennumber=2, sentencenumber=0, textid='maildir/zufferli-j/sent_items/99.'),
 Row(token='28618979.1075842030037.JavaMail.evans', tokennumber=3, sentencenumber=0, textid='maildir/zufferli-j/sent_items/99.')]

In [24]:
res = data.flatMap(tokenize_text).toDF()
print(type(res))
#print(res.count())
res.take(3)

<class 'pyspark.sql.dataframe.DataFrame'>


[Row(token='Message-ID', tokennumber=0, sentencenumber=0, textid='maildir/blair-l/personnel___promotions/1.'),
 Row(token=':', tokennumber=1, sentencenumber=0, textid='maildir/blair-l/personnel___promotions/1.'),
 Row(token='<', tokennumber=2, sentencenumber=0, textid='maildir/blair-l/personnel___promotions/1.')]